# Text Classification Notebook

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install PySastrawi
!pip install sklearn
!pip install keras-tuner

In [1]:
# Machine Learning related imports
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input

In [2]:
# Dataset and generic imports
import pandas as pd
import numpy as np
import sklearn
import os
import string

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [3]:
# Load dataset
filepath = os.getcwd()
datasetpath =  os.path.join(filepath, "cleaned_data", "laporanencoded.csv")

# Github URL for dataset when used in Google Colab
github_url = "https://raw.githubusercontent.com/Capstone-Project-B21-CAP0113/ml-tf/main/laporan/cleaned_data/laporanencoded.csv"

laporan = pd.read_csv(datasetpath, encoding="ISO-8859-1")
# Print dataset shape
print(laporan.shape)
# Print dataset head
laporan.head()


(2852, 27)


,text,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,mohon bantuannya untuk menormalkan sistem di l...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ass pak gubsaya perangkat desa karangsari kec ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,selamat sore bapak gubernur atau yang mewakili...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,assalamualaikum pak gub saya pengurus paguyuba...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,lapor saya okada arle sandi email okadaarlegm...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Split text and labels
label_list = [
    "perselisihan",
    "infrastruktur",
    "pemerintah",
    "kesehatan",
    "teknologi",
    "administrasi",
    "fasilitas",
    "lingkungan",
    "ketertiban",
    "listrik",
    "bahaya",
    "lainnya",
    "pungli",
    "ilegal",
    "lalulintas",
    "bencana",
    "air",
    "pendidikan",
    "kebersihan",
    "sosial",
    "wisata",
    "sara",
    "pencurian",
    "korupsi",
    "bbm",
    "keuangan"
] 
x = laporan["text"]
y = laporan[label_list]

# Print number of element in each category ( one element can have many label since it's a multi label classification )
for i in label_list:
    print("{}: {}".format(i, (laporan[i] == 1).sum()))

perselisihan: 16
infrastruktur: 353
pemerintah: 68
kesehatan: 80
teknologi: 23
administrasi: 693
fasilitas: 73
lingkungan: 114
ketertiban: 87
listrik: 43
bahaya: 22
lainnya: 139
pungli: 106
ilegal: 47
lalulintas: 42
bencana: 58
air: 85
pendidikan: 134
kebersihan: 9
sosial: 181
wisata: 13
sara: 7
pencurian: 6
korupsi: 39
bbm: 1
keuangan: 82


In [5]:
# Text head
x.head()

0    mohon bantuannya untuk menormalkan sistem di l...
1    ass pak gubsaya perangkat desa karangsari kec ...
2    selamat sore bapak gubernur atau yang mewakili...
3    assalamualaikum pak gub saya pengurus paguyuba...
4    lapor saya okada arle sandi email  okadaarlegm...
Name: text, dtype: object

In [6]:
# Label head
y.head()

,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,listrik,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Remove stopwords and stem words using Sastrawi
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stopword_factory = StopWordRemoverFactory()
stopword = stopword_factory.create_stop_word_remover()

for i in range(len(x)):
    x[i] = stopword.remove(x[i])
    x[i] = stemmer.stem(x[i])

x.head()

<ipython-input-7-7f4537c43ddc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = stopword.remove(x[i])
<ipython-input-7-7f4537c43ddc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = stemmer.stem(x[i])


0    bantu normal sistem lpse kabupaten demak mana ...
1    ass gubsaya perangkat desa karangsari kec kara...
2    selamat sore gubernur mewakilisaya ahmad rofiq...
3    assalamualaikum gub urus paguyuban dagang puja...
4    lapor okada arle sandi email okadaarlegmailcom...
Name: text, dtype: object

In [8]:
# Tokenize and pad text
vocab_size = 2000
embedding_dim = 16
max_length = 300
trunc_type = "post"
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x)

sequences = tokenizer.texts_to_sequences(x)

padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [9]:
# Shuffle, batch and separate data into train, dev and test
BUFFER_SIZE = 1000
BATCH_SIZE = 32
DATASET_SIZE = len(x)


tx = tf.convert_to_tensor(padded)
ty = tf.convert_to_tensor(y)

dataset = tf.data.Dataset.from_tensor_slices((tx, ty))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train_set = dataset.take(int(0.8 * len(dataset)))
test_set = dataset.skip(int(0.8 * len(dataset)))
test_set = dataset.take(int(0.2 * len(dataset))) 

print(len(train_set))
print(len(test_set))

72
18


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(26, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         256000    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         263168    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
NUM_EPOCHS = 100
cb = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.0001, patience=10)

history = model.fit(train_set, epochs=NUM_EPOCHS, validation_data=test_set, callbacks=[cb])

Epoch 1/200
72/72 [==============================] - 99s 1s/step - loss: 0.1272 - accuracy: 0.2331 - val_loss: 0.1166 - val_accuracy: 0.2361
Epoch 2/200
72/72 [==============================] - 76s 1s/step - loss: 0.1256 - accuracy: 0.2426 - val_loss: 0.1158 - val_accuracy: 0.2448
Epoch 3/200
72/72 [==============================] - 76s 1s/step - loss: 0.1202 - accuracy: 0.2873 - val_loss: 0.1075 - val_accuracy: 0.3524
Epoch 4/200
72/72 [==============================] - 75s 1s/step - loss: 0.1178 - accuracy: 0.3147 - val_loss: 0.1030 - val_accuracy: 0.3368
Epoch 5/200
72/72 [==============================] - 75s 1s/step - loss: 0.1134 - accuracy: 0.3424 - val_loss: 0.1007 - val_accuracy: 0.3438
Epoch 6/200
72/72 [==============================] - 74s 1s/step - loss: 0.1098 - accuracy: 0.3368 - val_loss: 0.0954 - val_accuracy: 0.3507
Epoch 7/200
72/72 [==============================] - 76s 1s/step - loss: 0.1061 - accuracy: 0.3451 - val_loss: 0.0954 - val_accuracy: 0.3351
Epoch 8/200
7

KeyboardInterrupt: 

In [18]:
model.save("/sm/classify_model")

INFO:tensorflow:Assets written to: /sm/classify_model\assets
INFO:tensorflow:Assets written to: /sm/classify_model\assets


In [19]:
# Text cleaning function for inferrence
def treatinput(inp):
    # Make string into lowercase string
    treated = inp.lower()
    # remove punctuation
    treated = treated.translate(str.maketrans("","",string.punctuation))
    # remove trailing whitespace
    treated = treated.strip()
    # Remove stopwords
    treated = stopword.remove(treated)
    # Stem string
    treated = stemmer.stem(treated)
    return treated

In [22]:
# sample inferrece
sample_laporan = "Oknum polisi di polsek sukajadi melakukan pungli"
treated_input = treatinput(sample_laporan)
print(treated_input)
pad = pad_sequences(tokenizer.texts_to_sequences([treated_input]), maxlen=max_length, truncating=trunc_type)
prediction = model.predict(pad)

res = dict(zip(label_list, prediction[0]))

oknum polisi polsek sukajadi pungli


In [23]:
# Print predictions
for i in res.keys():
    # if(res[i] > 0.6):
    print("{} {:.5f}".format(i, res[i]))

perselisihan 0.00000
infrastruktur 0.00008
pemerintah 0.00020
kesehatan 0.00099
teknologi 0.00000
administrasi 0.00167
fasilitas 0.00000
lingkungan 0.00000
ketertiban 0.00000
listrik 0.00000
bahaya 0.00000
lainnya 0.00000
pungli 0.90029
ilegal 0.00000
lalulintas 0.00000
bencana 0.00000
air 0.00000
pendidikan 0.00000
kebersihan 0.00000
sosial 0.00000
wisata 0.00000
sara 0.00000
pencurian 0.00000
korupsi 0.00010
bbm 0.00000
keuangan 0.00157


In [ ]:
# Try a new model with Keras Tuner // CAREFUL THIS TAKES A LONG TIME
import kerastuner as kt

In [ ]:
# Define Model
def model_builder(hp):
    emb_units = hp.Int('emb_units', min_value=16, max_value=256, step=16)
    bid1_units = hp.Int('bid1_units', min_value=16, max_value=256, step=16)
    drop1_frac = hp.Float('drop1_frac', min_value=0.1, max_value=0.8, step=0.1)
    bid2_units = hp.Int('bid2_units', min_value=16, max_value=256, step=16)
    drop2_frac = hp.Float('drop2_frac', min_value=0.1, max_value=0.8, step=0.1)
    dense1_units = hp.Int('dense1_unit', min_value=16, max_value=256, step=16)
    drop3_frac = hp.Float('drop3_frac', min_value=0.1, max_value=0.8, step=0.1)
    dense2_units = hp.Int('dense2_unit', min_value=16, max_value=256, step=16)

    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, emb_units),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(bid1_units, return_sequences=True)),
    tf.keras.layers.Dropout(drop1_frac),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(bid2_units)),
    tf.keras.layers.Dropout(drop2_frac),
    tf.keras.layers.Dense(dense1_units, activation='relu'),
    tf.keras.layers.Dropout(drop3_frac),
    tf.keras.layers.Dense(dense2_units, activation='relu'),
    tf.keras.layers.Dense(26, activation='sigmoid')
])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])
    
    return model

In [ ]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=100, factor=3, overwrite=True)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(train_set, epochs=100, validation_data=test_set, callbacks=[stop_early])

best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

In [ ]:
# Trial 5 Complete [00h 03m 30s]
# val_accuracy: 0.2395833283662796

# Best val_accuracy So Far: 0.2534722089767456
# Total elapsed time: 00h 55m 20s

# Search: Running Trial #6

# Hyperparameter    |Value             |Best Value So Far 
# emb_units         |48                |32                
# bid1_units        |240               |112               
# drop1_frac        |0.6               |0.3               
# bid2_units        |176               |112               
# drop2_frac        |0.4               |0.4               
# dense1_unit       |176               |112               
# drop3_frac        |0.7               |0.4               
# dense2_unit       |144               |224               
# learning_rate     |1e-05             |0.01              
# tuner/epochs      |2                 |2                 
# tuner/initial_e...|0                 |0                 
# tuner/bracket     |2                 |2                 
# tuner/round       |0                 |0                 

# Epoch 1/2
# 72/72 [==============================] - 618s 8s/step - loss: 0.6918 - accuracy: 0.0573 - val_loss: 0.6889 - val_accuracy: 0.0625
# Epoch 2/2
# 72/72 [==============================] - 623s 9s/step - loss: 0.6846 - accuracy: 0.0760 - val_loss: 0.6749 - val_accuracy: 0.0521

In [ ]:
model_t = tuner.hypermodel.build(best_hps)
history = model_t.fit(dataset, epochs=100, validation_data=test_set)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(dataset, epochs=best_epoch, validation_data=test_set)